In [1]:
import torch
import torchvision as tv
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import argparse

In [2]:
# 定义是否使用GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# 定义网络结构
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Sequential(     #input_size=(1*28*28)
            nn.Conv2d(1, 6, 5, 1, 2), #padding=2保证输入输出尺寸相同（28-5+2*2）/1+1=28
            #in_channels,out_channels,kernel_size,stride,padding
            nn.ReLU(),      #input_size=(6*28*28)
            nn.MaxPool2d(kernel_size=2, stride=2),#output_size=(6*14*14)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(6, 16, 5),#input_size=(6*14*14)(14-5+0*2)/1+1=10
            nn.ReLU(),      #input_size=(16*10*10)
            nn.MaxPool2d(2, 2)  #output_size=(16*5*5)
        )
        self.fc1 = nn.Sequential(
            nn.Linear(16 * 5 * 5, 120),#全连接层1#input_size=(16*5*5)
            #nn.linear(in_features,out_features)代表形状
            nn.ReLU()#output_szie=(120)
        )
        self.fc2 = nn.Sequential(
            nn.Linear(120, 84),#input_size=(120)
            nn.ReLU()#output_size=(84)
        )
        self.fc3 = nn.Linear(84, 10)#output: 10 possiblities
         # 定义前向传播过程，输入为x
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        
        x = x.view(x.size()[0], -1)# nn.Linear()的输入输出都是维度为一的值，所以把多维度的tensor展平成一维
       
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x


In [4]:
parser = argparse.ArgumentParser()
parser.add_argument('--outf', default='./model/', help='folder to output images and model checkpoints') 
#模型保存路径
parser.add_argument('--net', default='./model/net.pth', help="path to netG (to continue training)")  
#模型加载路径
opt = parser.parse_args(args=[])

In [5]:
# 超参数设置
EPOCH = 10   #遍历数据集、权重更新迭代次数,数据集多样化越强epoch越大。
#随着epoch 数量的增加， 权重更新迭代的次数增多， 曲线从最开始的不拟合状态， 进入优化拟合状态， 最终进入过拟合
BATCH_SIZE = 64      #批处理尺寸(batch_size)，每个batch中（在实际训练时、 将所有数据分成多个batch ， 每次送入一部分数据）训练样本的数量
LR = 0.001        #学习率

In [6]:
# 定义数据预处理方式
#变换图像数据为tensor,形状为[channel,height,width]
transform = transforms.ToTensor()

In [7]:
# 定义训练数据集
trainset = tv.datasets.MNIST(
    root='./data/',
    train=True,#读入数据为训练集
    download=True,#根目录root没有数据集，自动下载
    transform=transform #读入上一段的数据预处理方式
)

In [8]:
# 定义训练批处理数据
trainloader = torch.utils.data.DataLoader(
    dataset=trainset,#加载数据的数据集
    batch_size=BATCH_SIZE,#每个batch加载多少个样本
    shuffle=True,#默认为false，设置为true会在每个epoch重新打乱数据（鲁棒性、防止过拟合）
    )

In [9]:
# 定义测试数据集
testset = tv.datasets.MNIST(
    root='./data/',
    train=False,
    download=True,
    transform=transform)

In [10]:
# 定义测试批处理数据
testloader = torch.utils.data.DataLoader(
    dataset=testset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    )

In [11]:
# 定义损失函数loss function 和优化方式（采用SGD）
net = LeNet().to(device)
#损失函数
criterion = nn.CrossEntropyLoss()  # 交叉熵损失函数，通常用于多分类问题上
#优化器 SGD/Momentun/RMSprop/Adam
optimizer = optim.SGD(net.parameters(), lr=LR, momentum=0.9)
#(params, lr=<required parameter>, momentum=0, dampening=0, weight_decay=0, nesterov=False)

In [12]:
# 训练
if __name__ == "__main__":

    for epoch in range(EPOCH):
        sum_loss = 0.0
        # 数据读取
        for i, data in enumerate(trainloader):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            # 以前梯度清零
            optimizer.zero_grad()

            # forward + backward
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()#反向传播，计算当前梯度
            optimizer.step()#根据梯度更新网络参数

            # 每训练100个batch打印一次平均loss
            sum_loss += loss.item()
            if i % 100 == 99:
                print('[%d, %d] loss: %.03f'#loss保留三位小数
                      % (epoch + 1, i + 1, sum_loss / 100))
                sum_loss = 0.0
        # 每跑完一次epoch测试一下准确
        #所有计算得出的tensor的requires_grad都自动设置为False，反向传播不会自动求导
        with torch.no_grad():
            correct = 0
            total = 0
            for data in testloader:
                images, labels = data
                images, labels = images.to(device), labels.to(device)
                outputs = net(images)
                # 取得分最高的那个类
                _, predicted = torch.max(outputs.data, dim=1)#dim=1：输出所在行的最大值
                #torch.max返回两个值，第一个是具体的value（对应_),第二个是value对应的index（对应predicted）
                total += labels.size(0)
                correct += (predicted == labels).sum()
            print('第%d个epoch的识别准确率为：%d%%' % (epoch + 1, (100 * correct / total)))
    torch.save(net.state_dict(), '%s/net_%03d.pth' % (opt.outf, epoch + 1))

[1, 100] loss: 2.298
[1, 200] loss: 2.288
[1, 300] loss: 2.269
[1, 400] loss: 2.232
[1, 500] loss: 2.130
[1, 600] loss: 1.705
[1, 700] loss: 0.956
[1, 800] loss: 0.696
[1, 900] loss: 0.576
第1个epoch的识别准确率为：85%
[2, 100] loss: 0.450
[2, 200] loss: 0.421
[2, 300] loss: 0.387
[2, 400] loss: 0.360
[2, 500] loss: 0.323
[2, 600] loss: 0.307
[2, 700] loss: 0.301
[2, 800] loss: 0.302
[2, 900] loss: 0.248
第2个epoch的识别准确率为：93%
[3, 100] loss: 0.240
[3, 200] loss: 0.233
[3, 300] loss: 0.225
[3, 400] loss: 0.208
[3, 500] loss: 0.199
[3, 600] loss: 0.210
[3, 700] loss: 0.176
[3, 800] loss: 0.189
[3, 900] loss: 0.183
第3个epoch的识别准确率为：94%
[4, 100] loss: 0.171
[4, 200] loss: 0.158
[4, 300] loss: 0.152
[4, 400] loss: 0.162
[4, 500] loss: 0.147
[4, 600] loss: 0.155
[4, 700] loss: 0.137
[4, 800] loss: 0.151
[4, 900] loss: 0.129
第4个epoch的识别准确率为：96%
[5, 100] loss: 0.127
[5, 200] loss: 0.137
[5, 300] loss: 0.133
[5, 400] loss: 0.131
[5, 500] loss: 0.120
[5, 600] loss: 0.115
[5, 700] loss: 0.101
[5, 800] loss: 0.